**<h1>MACHINE LEARNING</h1>**

1 - unir as colunas de respostas
2 - catalogar as labels um por um.
3 - usar word2vec para vetorizar os tweets
4 - aplicar tecnicas de ml
5 - checar resultados
6 - aplicar tecnicas que melhorem os resultados
7 - avaliação.

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('vacine_tweets_completoDezJanFevTratado.csv')

In [4]:
df

,Apelido,Username,Data/Hora,Respondendo,Texto,link_img,Comentarios,Retweets,Likes
0,Eder Morselli,@edermorselli,2020-11-30T23:54:49.000Z,A pandemia não acabou... Vamos nos proteger......,NaN,https://pbs.twimg.com/media/EoHB4fVXMAEdkk5?fo...,0,1,1
1,V,@VR_SSC,2020-11-30T23:53:24.000Z,Em resposta a \n@KooperHD,Os caras tacam capa variante logo quando a kug...,https://abs-0.twimg.com/emoji/v2/72x72/1f643.png,0,0,2
2,Lucas França,@FrancaComAmor,2020-11-30T23:53:32.000Z,Publico na \n@RevistaEpoca\n um papo que tive ...,Covid-19: Quem deve receber primeiro a vacina?...,https://pbs.twimg.com/card_img/136204193687451...,0,6,27
3,calcinha chromatica,@fckwithmyselff,2020-11-30T23:52:55.000Z,eu só penso nela (a vacina),NaN,https://pbs.twimg.com/media/EoHBcUKWEAEcXHx?fo...,1,4,4
4,Rafa_SCH,@RafaSCHk,2020-11-30T23:54:43.000Z,"Difícil, mas acho que pegaria a variante",Comentar o Tweet\nKooper // 110K no Youtube\n@...,https://pbs.twimg.com/profile_images/129763947...,0,0,1
...,...,...,...,...,...,...,...,...,...
2120,vını,@vnftn,2021-02-19T23:29:46.000Z,Em resposta a \n@todasemanaum\n @LOUDgg\n e ma...,Quando corona passar vai ter copa do mundo val...,https://abs-0.twimg.com/emoji/v2/72x72/1f976.png,0,0,0
2121,UOL Notícias,@UOLNoticias,2021-02-19T22:12:22.000Z,"Com UTIs cheias, interior de SP transfere paci...","Com UTIs cheias, interior de SP transfere paci...",https://pbs.twimg.com/card_img/136288568773806...,8,15,76
2122,n_rto,@n_rto,2021-02-19T23:13:19.000Z,O Antagonista: deputado defende abertura de es...,Comentar o Tweet\nGabriel Azevedo\n@gabrielaze...,https://pbs.twimg.com/profile_images/132857609...,0,0,0
2123,𝐬𝐭𝐞⁹⁹☂︎,@gellerscss,2021-02-19T23:52:38.000Z,Em resposta a \n@SeriesTWBZ,depois de tomar vacina #KCA\n #VoteCobraKai,https://abs.twimg.com/hashflags/Nickelodeon_KC...,0,0,0


In [5]:
df_x = df[["Respondendo", "Texto"]].copy()

In [6]:
df_x.isnull().sum()

Respondendo      0
Texto          982
dtype: int64

In [7]:
df_x['Texto'] = df_x['Texto'].fillna("")

In [8]:
df_x["tweet"] = df_x["Respondendo"] + " " + df_x["Texto"]

In [9]:
df_t = df_x.drop(columns=['Respondendo', 'Texto'])

In [9]:
df_t.to_csv('tweetsReduzidos.csv', index=None,encoding='utf-8')

In [114]:
df_t.to_excel('tweetsReduzidos1.xlsx', index=None)

# PREPROCESSING

In [13]:
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import gensim

import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

C:\Users\romul\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [11]:
df_t.shape

(2125, 1)

In [10]:
df_t = pd.read_csv('tweetsReduzidos.csv')

In [11]:
df_t

,tweet
0,A pandemia não acabou... Vamos nos proteger......
1,Em resposta a \n@KooperHD Os caras tacam capa ...
2,Publico na \n@RevistaEpoca\n um papo que tive ...
3,eu só penso nela (a vacina)
4,"Difícil, mas acho que pegaria a variante Comen..."
...,...
2120,Em resposta a \n@todasemanaum\n @LOUDgg\n e ma...
2121,"Com UTIs cheias, interior de SP transfere paci..."
2122,O Antagonista: deputado defende abertura de es...
2123,Em resposta a \n@SeriesTWBZ depois de tomar va...


### BIGRAMAS

In [79]:
from gensim.models.phrases import Phrases, Phraser

In [80]:
sent = [row.split() for row in df_t['tweet']]

In [81]:
phrases = Phrases(sent, min_count=30, progress_per=10000)

In [82]:
bigram = Phraser(phrases)

In [83]:
sentences = bigram[sent]

In [26]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

17983

In [27]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['de', 'a', 'e', 'que', 'o', 'da', 'do', 'vacina', 'em', 'para']

### TREINANDO O MODELO

In [84]:
import multiprocessing

from gensim.models import Word2Vec

In [85]:
cores = multiprocessing.cpu_count()
cores

4

In [86]:
w2v_model = Word2Vec(min_count=2,
                     window=5,
                     vector_size=10,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1,
                    epochs=50)

In [87]:
w2v_model.save("word2vec.wordvectors")

In [88]:
w2v_model.build_vocab(sentences, progress_per=10000)

In [89]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(930197, 2300460)

In [34]:
vector = w2v_model.wv['bolsonaro']
vector

array([ 0.3477075 , -0.8539045 ,  0.43761876, -0.02841874,  0.72045696,
        1.104511  , -0.18680568,  1.4620262 , -1.260303  ,  0.30498236],
      dtype=float32)

In [37]:
vec = w2v_model.wv['bolsonaro']

In [38]:
w2v_model.wv.most_similar(positive=[vec])

[('bolsonaro', 1.0),
 ('conhecimento', 0.9991538524627686),
 ('bolsominion', 0.9990280866622925),
 ('locais', 0.9987854957580566),
 ('UBS', 0.9985707998275757),
 ('debate', 0.9984819293022156),
 ('si', 0.9983919262886047),
 ('chamada', 0.9983118772506714),
 ('#sp2', 0.9982222318649292),
 ('amigo', 0.9981968998908997)]

In [42]:
w2v_model.wv.similarity('bolsominion', 'vacina')

0.9546063

### TREINANDO COM DOC2VEC

In [90]:
import gensim

In [91]:
def read_corpus(df_t, tokens_only=False):
    for i in df_t["tweet"]:
        tokens = gensim.utils.simple_preprocess(i)
        if tokens_only:
            yield tokens
        else:
            yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

In [92]:
corpus = list(read_corpus(df_t))

In [93]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=10, min_count=2, epochs=40)

In [94]:
model.build_vocab(corpus)

In [95]:
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [96]:
vector = model.infer_vector(['A', 'pandemia', 'não', 'acabou'])
print(vector)

[ 0.05862779  0.00162674  0.02037181 -0.08301528  0.03194065  0.07401339
  0.48601267  0.20121534 -0.69579995 -0.19577397]


In [97]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\romul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [98]:
df_y.head(15)

,tweet,y
0,[-0.16756459 -0.25380713 0.00172649 -0.083690...,0
1,[-0.0141285 -0.09096924 -0.49570632 -0.094470...,0
2,[-0.42473194 -0.5366614 -0.03413643 -0.356889...,0
3,[ 0.06222841 0.02009614 0.05173865 0.139120...,0
4,[-1.5607463e-02 7.8439993e-01 -1.3763020e-02 ...,0
5,[ 0.1437726 -0.00832112 0.24646358 -0.049779...,0
6,[ 5.4742046e-02 7.1585990e-02 -6.6739537e-02 ...,0
7,[ 0.23935327 -0.27494898 0.3203835 0.308783...,0
8,[-0.10997692 0.3096803 0.0071757 -0.062270...,0
9,[-0.39370605 0.22923796 0.4451299 -0.049135...,0


In [37]:
for i in range(len(df_y)):
    sentence = str(df_y["tweet"][i])
    tokens = nltk.word_tokenize(sentence)
    tokens = list(tokens)
    vector = model.infer_vector(tokens)
    df_y["tweet"][i] = vector

In [ ]:
df_y.to_csv('tweetsClassificadosTransformados.csv', index=None,encoding='utf-8')

In [71]:
df = pd.read_csv('tweetsClassificadosTransformados.csv', error_bad_lines=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2108 entries, 0 to 2107
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   2108 non-null   object
 1   y       2108 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 33.1+ KB


In [72]:
df.head(15)

,tweet,y
0,[-0.16756459 -0.25380713 0.00172649 -0.083690...,0
1,[-0.0141285 -0.09096924 -0.49570632 -0.094470...,0
2,[-0.42473194 -0.5366614 -0.03413643 -0.356889...,0
3,[ 0.06222841 0.02009614 0.05173865 0.139120...,0
4,[-1.5607463e-02 7.8439993e-01 -1.3763020e-02 ...,0
5,[ 0.1437726 -0.00832112 0.24646358 -0.049779...,0
6,[ 5.4742046e-02 7.1585990e-02 -6.6739537e-02 ...,0
7,[ 0.23935327 -0.27494898 0.3203835 0.308783...,0
8,[-0.10997692 0.3096803 0.0071757 -0.062270...,0
9,[-0.39370605 0.22923796 0.4451299 -0.049135...,0


#### Transformar tipos

In [99]:
count=0
for i in range(len(df_y)):
    a


NameError: name 'a' is not defined

### Começando a aplicar machine learning